<a href="https://colab.research.google.com/github/carloshfmaciel/datascience/blob/master/Sistema_de_Recomenda%C3%A7%C3%A3o_SVD_Livro_Machine_Learning_Make_Your_Own_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Recomendando Livros - Filtragem Colaborativa baseada em itens

## Sobre Mim

Carlos Maciel trabalha com desenvolvimento de software há 12 anos, tendo atuado em diversos projetos em empresas pública e privadas, incluindo também o setor financeiro.

Atualmente, como Tech Lead na Superbid, tem implementado soluções de data science visando o aumento de valor de vendas no Superbid Marketplace.

**Contatos**:<br>
Email: carloshfmaciel@gmail.com<br>
Linkedin: https://www.linkedin.com/in/carloshfmaciel/

##Introdução

Esse notebook tem o objetivo de consolidar através do exercício e explanação detalhada o conteúdo do exercício **Recommending Books do livro Machine Learning: Make Your Own Recommender System** de **Oliver Theobald**, que pode ser adquirido em livrarias virtuais como a Amazon.

## Sistemas de Recomendação

Atualmente, a maioria das ferramentas de ecommerce ou serviços digitais fazem algum tipo de recomendação ao usuário.

Seja em um site de e-commerce, que sugere aquele produto que você vai querer comprar ou a Netflix que recomenda aquele filme que você pode gostar.

Para fazer isso, esses sites possuem algoritmos que analisam suas características e comportamentos, como também dos demais usuários, indentificando assim possíveis padrões ou semelhanças, recomendando assim algo que tem grandes chances de você querer/gostar.

## Filtragem Colaborativa

O filtro que possibilita essa recomendação é chamado de **Filtragem Colaborativa**, ou seja, um filtro construído a partir da colaboração entre os usuários daquele sistema.

Dentro da filtragem colaborativa, temos uma técnica que é chamada de **Filtragem Colaborativa baseada no item**, que obtém itens associados á um determinado item de interesse do usuário. Essa associação de um item com outros itens, se dá através das preferências/avaliação/ação de outros usuários.

Ou seja, pega-se um item que um usuário comprou/clicou/gostou, verifica todos os usuários que compraram/clicaram/gostaram desse item e recupera outros itens que esses usuários também compraram/clicaram/gostaram.

Para elucidar essa técnica, iremos trabalhar com dados referente á avaliações/notas que usuários deram para livros.

Baseado nessas avaliações, iremos aplicar o cálculo de correlação, para identificarmos quais livros possuem forte relação com um determinado livro e assim podermos recomendar livros para um determinado usuário.

Ou seja, um usuário acessa o sistema, compra ou clica em um livro, identificado esse livro, identificamos quais outros livros que estão correlacionados com este através das preferências/avaliações dos outros usuários.

## Algoritmo

No nosso exemplo, o dataset é *esparso*, ou seja, na intersecção **ratings de usuários sobre filmes**, temos vários filmes que não foram avaliados por vários usuários, portanto nessa intersecção temos um valor nulo.

Geralmente os algoritmos não trabalham bem com valores nulos, portanto devemos atribuir um valor numérico, que nesta análise serão substituidos por 0.

Para trabalhar com datasets esparso um algoritmo recomendando é o **Singular Value Decomposition** ou **SVD**.

## Dataset

O dataset utilizado neste exercício fora coletado pelo www.goodreads.com e baixado pelo github https://github.com/zygmuntz/goodbooks-10k.

Esse dataset possui 10K livros e mais de 5 milhões de avaliações de usuários sobre esses livros.

Neste exemplo estaremos utilizando dois datasets: **books.csv e ratings.csv**.

##Codificando

In [0]:
# imports necessários
import numpy as np
import pandas as pd
from sklearn.decomposition import TruncatedSVD

In [0]:
# load datasets
ratings = pd.read_csv("https://dl.dropboxusercontent.com/s/93l02j2xe1mtoxk/ratings.csv?dl=0")

books = pd.read_csv("https://dl.dropboxusercontent.com/s/bl2t73fgtov43td/books.csv?dl=0")

Como o nosso dataset ratings possui mais de 5 milhões de avaliações, precisamos diminuir nosso dataset por limitações de processamento, uma vez que estamos rodando no Google Colab. Para isto estaremos extraindo uma amostra aleatória do mesmo de 100K avaliações.

In [0]:
# Pelo fato do dataset ser muito grande, extrairemos do mesmo uma amostra aleatoria de 100K
ratings_sample = ratings.sample(100000)

Abaixo estaremos juntando os dois datasets utilizando como parâmetro da relação entre ambos, o id do livro, propriedade esta presente em ambos.

In [5]:
# merge dataset
combined_books_data = pd.merge(ratings_sample, books, on="book_id")

# Listando as 5 primeiras linhas
combined_books_data.head()

,user_id,book_id,rating,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,28669,474,4,14743,14743,3044365,104,618680004,9.780619e+12,Richard Dawkins,2006.0,The God Delusion,The God Delusion,eng,3.89,162740,176774,6812,9571,11019,31723,60769,63692,https://images.gr-assets.com/books/1347220693m...,https://images.gr-assets.com/books/1347220693s...
1,7582,474,3,14743,14743,3044365,104,618680004,9.780619e+12,Richard Dawkins,2006.0,The God Delusion,The God Delusion,eng,3.89,162740,176774,6812,9571,11019,31723,60769,63692,https://images.gr-assets.com/books/1347220693m...,https://images.gr-assets.com/books/1347220693s...
2,52720,474,4,14743,14743,3044365,104,618680004,9.780619e+12,Richard Dawkins,2006.0,The God Delusion,The God Delusion,eng,3.89,162740,176774,6812,9571,11019,31723,60769,63692,https://images.gr-assets.com/books/1347220693m...,https://images.gr-assets.com/books/1347220693s...
3,41626,474,4,14743,14743,3044365,104,618680004,9.780619e+12,Richard Dawkins,2006.0,The God Delusion,The God Delusion,eng,3.89,162740,176774,6812,9571,11019,31723,60769,63692,https://images.gr-assets.com/books/1347220693m...,https://images.gr-assets.com/books/1347220693s...
4,39802,474,4,14743,14743,3044365,104,618680004,9.780619e+12,Richard Dawkins,2006.0,The God Delusion,The God Delusion,eng,3.89,162740,176774,6812,9571,11019,31723,60769,63692,https://images.gr-assets.com/books/1347220693m...,https://images.gr-assets.com/books/1347220693s...


In [123]:
combined_books_data.shape

(100000, 25)

Para realizar o cálculo do coefiente de correlação de cada uma dos livros com todos os demais, baseado nas preferências dos usuários(nas notas) em relação aos livros precisaremos criar uma tabela pivot.

Uma tabela pivot é uma matriz que contém em suas dimensões apenas os dados relevantes para o cálculo de correlação, portanto precisamos do id do usuário, do título do livro e na intersecção entre essas duas dimensões, a nota do usuário para o livro.

Por questões do algoritmo SVD, precisamos também substituir os valores nulos/inexistentes por 0.

Abaixo, criação da tabela pivot:

In [6]:
# Criando a utility matrix(pivot table)
rating_utility_matrix = combined_books_data.pivot_table(index="user_id", columns="title", values="rating", fill_value=0)

rating_utility_matrix.head()

title,"Angels (Walsh Family, #3)","""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",'Salem's Lot,"'Tis (Frank McCourt, #2)","1,000 Places to See Before You Die",1/4 جرام,"10% Happier: How I Tamed the Voice in My Head, Reduced Stress Without Losing My Edge, and Found Self-Help That Actually Works","100 Bullets, Vol. 1: First Shot, Last Call",100 Love Sonnets,100 Selected Poems,"10th Anniversary (Women's Murder Club, #10)","11 Birthdays (Willow Falls, #1)",11/22/63,"11th Hour (Women's Murder Club, #11)","12th of Never (Women's Murder Club, #12)","13 Gifts (Willow Falls, #3)",13 Hours: The Inside Account of What Really Happened In Benghazi,"13 Little Blue Envelopes (Little Blue Envelope, #1)",14,1421: The Year China Discovered America,1491: New Revelations of the Americas Before Columbus,1493: Uncovering the New World Columbus Created,"14th Deadly Sin (Women’s Murder Club, #14)",15th Affair (Women's Murder Club #15),"16 Lighthouse Road (Cedar Cove, #1)",1776,1919,1984,1Q84,"1Q84 #1-2 (1Q84, #1-2)","1Q84 BOOK 1 (1Q84, #1)","1Q84 BOOK 2 (1Q84, #2)","1Q84 BOOK 3 (1Q84, #3)","1st to Die (Women's Murder Club, #1)",2 States: The Story of My Marriage,2 ضباط,"2001: A Space Odyssey (Space Odyssey, #1)","2010: Odyssey Two (Space Odyssey, #2)","204 Rosewood Lane (Cedar Cove, #2)","206 Bones (Temperance Brennan, #12)",...,حوجن [Ḥawjan],حول العالم في 200 يوم,درخت زیبای من,ذاكرة الجسد,رأيت رام الله,رباعيات صلاح جاهين,رجال في الشمس,رحلتي من الشك إلى الإيمان,رغم الفراق,زغازيغ,ساق البامبو,سمفونی مردگان,سینوهه,شكلها باظت,شيكاجو,صانع الظلام,طه الغريب,ظل الأفعى,عائد إلى حيفا,عابر سرير,عزازيل,عقاید یک دلقک,فلتغفري,فوضى الحواس,في ديسمبر تنتهي كل الأحلام,في قلبي أنثى عبرية,قمر على سمرقند,كخه يا بابا,لا تحزن,ليتها تقرأ,ماهی سیاه کوچولو,محال,مخطوطة بن إسحاق: مدينة الموتى,نادي السيارات,هيبتا,واحة الغروب,يوتوبيا,ڤيرتيجو,キスよりも早く1 [Kisu Yorimo Hayaku 1] (Faster than a Kiss #1),美少女戦士セーラームーン新装版 1 [Bishōjo Senshi Sailor Moon Shinsōban 1]
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Quando criamos a tabela pivot, qualquer dimensão(no caso usuario e livro), que não tenha pelo menos uma intersecção(valor diferente de 0) com a outra dimensão, é descartado da tabela.

Sendo assim, nos nossos datasets originais temos **100K de usuários/avaliações** e **10K de books**.

Porém após a construção da tabela pivot, vemos que a mesma possui apenas 44546 linhas(usuários/avaliações) e 9487 colunas(livros).

Ou seja, dos 100K usuários, mais de 55K não fizeram avaliação em pelo menos 1 dos 10K livros do dataset.

E dos 10K de livros, mais de 500 livros não receberam avaliação de nenhum dos 100K de usuários.

In [7]:
rating_utility_matrix.shape

(44546, 9487)

Na análise que estamos realizando, queremos saber **a correlação existente entre os livros**, baseado nas preferências do usuário. Portanto nossa variável de resposta são os livros, ou seja nosso eixo Y.

Sabemos que o eixo Y é representado por linhas e o X por colunas.

Porém, na nossa tabela pivot os Livros está dispostos em colunas, ou seja eixo X, portanto temos que inverter, ou seja, transpor os livros em linhas e os usuários em colunas.

Fazemos isso, através do método abaixo, o que chamamos de transposição de matriz.

In [0]:
# Transpondo Linhas em Colunas e Colunas em Linhas, ou seja, usuário como colunas e filmes como linhas
X = rating_utility_matrix.T

Como podemos ver abaixo, nossa tabela pivot foi transposta, sendo agora o nosso eixo Y(linhas), os livros:

In [125]:
X.head()

user_id,1,3,4,5,6,7,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,43,...,53373,53375,53376,53377,53378,53380,53381,53382,53383,53384,53385,53386,53388,53389,53390,53393,53394,53395,53398,53399,53400,53401,53402,53403,53404,53407,53409,53410,53411,53412,53413,53415,53416,53418,53419,53420,53421,53422,53423,53424
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"Angels (Walsh Family, #3)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""حكايات فرغلي المستكاوي ""حكايتى مع كفر السحلاوية",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
'Salem's Lot,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"'Tis (Frank McCourt, #2)",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"1,000 Places to See Before You Die",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Abaixo, para reduzir/comprimir os valores(notas) dos usuários individuais, totalizando 44.546, iremos através do algoritmo SVD, converter esse número total de avaliações em apenas 30 componentes(30 colunas).

In [9]:
# Comprimir os milhares de usuários em 30
SVD = TruncatedSVD(n_components=30)
transposed_matrix = SVD.fit_transform(X)

(9487, 30)

Perceba que agora nossa matriz com as colunas/features comprimidas, possui de 44.546 colunas para apenas 30.

In [126]:
transposed_matrix.shape

(9487, 30)

Agora, iremos calcular o coeficiente de correlação de cada um dos 9487 livros com todos os demais, baseado nas preferências/avaliações de cada usuário para cada livro.

In [77]:
# Calculando a correlação de cada um dos livros com todos os demais, ou seja, o usuário 
# Lembrando aqui que meu X(Colunas) é a nota do usuario para o filme e o meu Y(Linhas) são os filmes
corr_matrix = np.corrcoef(transposed_matrix)

(9487, 9487)

Perceba que a matriz com os resultados dos coefientes de correlação, possui uma dimensão diferente da matriz resultado da compressão(SVD) anterior. A matriz anterior possui (9487 x 30). A matriz com os coeficientes possui (9487 x 9487).

Isso ocorre pois agora temos nessa matriz os livros tanto nas linhas quanto nas colunas, ou seja, a correlação de cada livro com todos os demais, inclusive com ele mesmo, e na interseção temos o coeficiente de correlação entre o livro da linha com o livro da coluna.

In [127]:
# O resultado é uma matriz do tamanho quantidade livros(linhas) pela quantidade de livros(colunas)
corr_matrix.shape

(9487, 9487)

Para o nosso exemplo, vamos supor, que o usuário para o qual queremos recomendar livros, tenhamos identificado que ele comprou ou gosta do **livro 1984 de George Orwell**.

Iremos identificar qual posição de índice o mesmo se encontra na listagem de livros.

In [82]:
# Obtendo a posição/índice no dataset mergeado se encontra o Livro 1984 de George Orwell

books = rating_utility_matrix.columns
books_list = list(books)

book_1984 = books_list.index('1984')
book_1984

book_1984

27

Uma vez identificado a posição do mesmo, podemos buscar na matriz que possui os valores de correlação, todos os valores de correlação deste livro(1984 de George Orwell) com todos os demais livros.

In [85]:
# Obtendo a correlação calculada para o Livro 1984 de George Orwell
# A mesma posicao numerica que o livro tinha quando era coluna, continua tendo como linha

corr_1984 = corr_matrix[27]

# Aqui temos um 9487 coeficientes de correlação do Livro George Orwell com cada um dos 9487, incluso ele mesmo
corr_1984

array([-0.04061331,  0.00421706,  0.54327192, ...,  0.2928387 ,
        0.03644   ,  0.09006144])

Com os valores de coeficiente de correlação da relação do livro 1984 do Gerge Orwell com todos os demais, listamos todos os livros que possuem uma correlação forte com ele, ou seja, cujo valor da correlação seja superior á 0.8.

Dessa forma, temos uma lista de livros que baseado nas preferências/avaliações do usuários, tem grande chance do usuário para o qual queremos recomendar, gostar.

In [122]:
# Imprimir lista de livros que possuem score de correlação entre 0.8 e 1.0
list(books[(corr_1984 < 1.0) & (corr_1984 > 0.8)])

['61 Hours (Jack Reacher, #14)',
 'A Corner of the Universe',
 'A Long Way Gone: Memoirs of a Boy Soldier',
 'An Artist of the Floating World',
 'As You Wish: Inconceivable Tales from the Making of The Princess Bride',
 'Batman and Son',
 "Batman: No Man's Land, Vol. 1",
 'Batman: The Dark Knight Returns (The Dark Knight Saga, #1)',
 'Beatrice and Virgil',
 'Black Butler, Vol. 2 (Black Butler, #2)']